In [1]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
df = pd.read_csv('../data/curated/Final_Merged_Dataset.csv')


unwanted_years = [str(i) for i in range(2000, 2012)]


for years in unwanted_years:
    df = df[df.columns.drop(list(df.filter(regex=years)))]
    
immigration_columns = ['cost_text', 'bedrooms', 'bathrooms', 'parking_spaces' ,'2016_immigration',
 '2017_immigration',
 '2018_immigration',
 '2019_immigration',
 '2020_immigration',
 '2021_immigration']

df = df.drop(immigration_columns, axis = 1 )
df = df.groupby('suburb', as_index = False).mean()

df

,suburb,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,2012_median_rent,2013_median_rent,2014_median_rent,2015_median_rent,2016_median_rent,...,2012_unemployment_rate,2013_unemployment_rate,2014_unemployment_rate,2015_unemployment_rate,2016_unemployment_rate,2017_unemployment_rate,2018_unemployment_rate,2019_unemployment_rate,2020_unemployment_rate,2021_unemployment_rate
0,abbotsford,2.934439,1.489185,4.443090,1.067517,450.0,470.0,425.0,440.0,450.0,...,5.3,5.5,5.4,4.5,3.4,4.1,2.9,3.1,4.7,4.5
1,albert park,44.218611,11.892911,5.411578,0.473433,470.0,460.0,480.0,500.0,520.0,...,2.3,2.7,3.2,2.4,2.2,2.1,1.6,2.1,3.6,2.7
2,alfredton,4.955048,6.510578,121.322730,1.698581,270.0,280.0,280.0,285.0,290.0,...,2.8,2.2,2.6,2.6,2.4,2.0,2.2,1.5,3.4,2.2
3,alphington,3.177273,1.260436,9.481345,1.014145,330.0,350.0,350.0,345.0,375.0,...,4.4,5.9,5.7,5.1,4.2,4.7,3.8,4.2,5.8,4.9
4,altona,1.313035,0.931396,18.746987,0.597719,330.0,340.0,350.0,360.0,375.0,...,4.7,4.5,4.7,5.0,5.2,5.7,4.3,3.5,5.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,wendouree,2.355989,2.664463,120.395226,0.755221,270.0,280.0,280.0,285.0,290.0,...,8.9,7.3,8.9,7.7,8.1,6.2,6.9,6.0,9.0,6.2
149,west footscray,2.163598,1.444609,10.739275,1.967266,320.0,320.0,330.0,340.0,360.0,...,6.6,6.2,6.4,6.7,6.7,7.4,5.6,5.2,7.7,5.4
150,williamstown,1.569507,0.901973,14.134295,0.712710,440.0,450.0,440.0,461.0,490.0,...,3.4,3.0,3.7,3.4,3.6,3.9,3.0,2.6,4.2,2.7
151,wodonga,2.278371,4.705043,312.975948,1.161629,265.0,270.0,285.0,290.0,290.0,...,7.0,7.1,6.4,7.5,6.6,4.3,4.8,5.2,4.2,3.9


In [3]:
median_rents = ['suburb', '2012_median_rent', '2013_median_rent', '2014_median_rent', 
                 '2015_median_rent', '2016_median_rent', '2017_median_rent',  
                 '2018_median_rent', '2019_median_rent', '2020_median_rent', 
                 '2021_median_rent']

incomes = ['suburb', '2012_annual_income', '2013_annual_income', '2014_annual_income',
           '2015_annual_income', '2016_annual_income', '2017_annual_income',
           '2018_annual_income', '2019_annual_income', '2020_annual_income', 
           '2021_annual_income']

populations = ['suburb', 'population_2012', 'population_2013', 'population_2014', 
               'population_2015', 'population_2016', 'population_2017', 
               'population_2018', 'population_2019', 'population_2020', 
               'population_2021']

population_density = ['suburb', 'population_density_2012', 'population_density_2013', 'population_density_2014',
                 'population_density_2015', 'population_density_2016',
                 'population_density_2017', 'population_density_2018',
                 'population_density_2019', 'population_density_2020',
                 'population_density_2021']

unemployement = ['suburb', '2012_unemployment_rate', '2013_unemployment_rate', '2014_unemployment_rate',
                 '2015_unemployment_rate', '2016_unemployment_rate', '2017_unemployment_rate', 
                 '2018_unemployment_rate', '2019_unemployment_rate', '2020_unemployment_rate',
                 '2021_unemployment_rate']

In [4]:
melt_rents = pd.melt(df[median_rents], id_vars = 'suburb')
melt_rents['year'] = pd.to_numeric(melt_rents['variable'].str[0:4]) 
melt_rents = melt_rents.drop('variable', axis = 1) 
melt_rents = melt_rents.rename(columns = {'value' : 'median_rent'})

all_rents = melt_rents['median_rent']

melt_rents

,suburb,median_rent,year
0,abbotsford,450.000000,2012
1,albert park,470.000000,2012
2,alfredton,270.000000,2012
3,alphington,330.000000,2012
4,altona,330.000000,2012
...,...,...,...
1525,wendouree,357.208581,2021
1526,west footscray,387.983260,2021
1527,williamstown,516.017635,2021
1528,wodonga,329.083134,2021


In [5]:
melt_income = pd.melt(df[incomes], id_vars = 'suburb')
melt_income['year'] = pd.to_numeric(melt_income['variable'].str[0:4]) 
melt_income = melt_income.drop('variable', axis = 1) 
melt_income = melt_income.rename(columns = {'value' : 'mean_income'})

all_incomes = melt_income['mean_income']

melt_income

,suburb,mean_income,year
0,abbotsford,65938.000000,2012
1,albert park,107915.000000,2012
2,alfredton,52993.000000,2012
3,alphington,68272.000000,2012
4,altona,56677.000000,2012
...,...,...,...
1525,wendouree,50455.929203,2021
1526,west footscray,60293.192444,2021
1527,williamstown,91634.348140,2021
1528,wodonga,54696.818254,2021


In [6]:
melt_population = pd.melt(df[populations], id_vars = 'suburb')
melt_population['year'] = pd.to_numeric(melt_population['variable'].str[11:]) 
melt_population = melt_population.drop('variable', axis = 1) 
melt_population = melt_population.rename(columns = {'value' : 'population'})

all_populations = melt_population['population']

melt_population

,suburb,population,year
0,abbotsford,5544.0,2012
1,albert park,15142.0,2012
2,alfredton,9060.0,2012
3,alphington,8706.0,2012
4,altona,12361.0,2012
...,...,...,...
1525,wendouree,15216.0,2021
1526,west footscray,12800.0,2021
1527,williamstown,17065.0,2021
1528,wodonga,27569.0,2021


In [7]:
melt_population_density = pd.melt(df[population_density], id_vars = 'suburb')
melt_population_density['year'] = pd.to_numeric(melt_population_density['variable'].str[19:]) 
melt_population_density = melt_population_density.drop('variable', axis = 1) 
melt_population_density = melt_population_density.rename(columns = {'value' : 'population_density'})

all_population_densitu = melt_population_density['population_density']

melt_population_density

,suburb,population_density,year
0,abbotsford,3185.291583,2012
1,albert park,3239.138340,2012
2,alfredton,171.880306,2012
3,alphington,3017.363879,2012
4,altona,698.965773,2012
...,...,...,...
1525,wendouree,225.177363,2021
1526,west footscray,2140.217700,2021
1527,williamstown,2322.938078,2021
1528,wodonga,98.879859,2021


In [8]:
melt_unemployement = pd.melt(df[unemployement], id_vars = 'suburb')
melt_unemployement['year'] = pd.to_numeric(melt_unemployement['variable'].str[0:4]) 
melt_unemployement = melt_unemployement.drop('variable', axis = 1) 
melt_unemployement = melt_unemployement.rename(columns = {'value' : 'unemployement'})

all_unemployment = melt_unemployement['unemployement']

melt_unemployement

,suburb,unemployement,year
0,abbotsford,5.3,2012
1,albert park,2.3,2012
2,alfredton,2.8,2012
3,alphington,4.4,2012
4,altona,4.7,2012
...,...,...,...
1525,wendouree,6.2,2021
1526,west footscray,5.4,2021
1527,williamstown,2.7,2021
1528,wodonga,3.9,2021


In [9]:
melted_dfs = [melt_rents, melt_income, melt_population, melt_population_density, melt_unemployement ]
modelling_dataframe = pd.concat(melted_dfs, join='inner', axis=1)
modelling_dataframe = modelling_dataframe.loc[:,~modelling_dataframe.columns.duplicated()].copy()

modelling_dataframe

,suburb,median_rent,year,mean_income,population,population_density,unemployement
0,abbotsford,450.000000,2012,65938.000000,5544.0,3185.291583,5.3
1,albert park,470.000000,2012,107915.000000,15142.0,3239.138340,2.3
2,alfredton,270.000000,2012,52993.000000,9060.0,171.880306,2.8
3,alphington,330.000000,2012,68272.000000,8706.0,3017.363879,4.4
4,altona,330.000000,2012,56677.000000,12361.0,698.965773,4.7
...,...,...,...,...,...,...,...
1525,wendouree,357.208581,2021,50455.929203,15216.0,225.177363,6.2
1526,west footscray,387.983260,2021,60293.192444,12800.0,2140.217700,5.4
1527,williamstown,516.017635,2021,91634.348140,17065.0,2322.938078,2.7
1528,wodonga,329.083134,2021,54696.818254,27569.0,98.879859,3.9


## Step 1 - Create Test Set for Future 

* Use ARMA to determine median_rent, mean_income, population, population_density and unemployement for all suburbs , for the years 2022, 2023, 2024


In [10]:
def arma_preds(attribute):
    
    sa2_list = [ i for i in modelling_dataframe['suburb'].unique()]
    full_predictions = pd.DataFrame(columns = ['suburb', attribute , 'year'])
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        for i in sa2_list:

            curr_suburb = modelling_dataframe[modelling_dataframe['suburb'] == i]

            from statsmodels.tsa.statespace.sarimax import SARIMAX

            y = curr_suburb[attribute]

            ARMAmodel = SARIMAX(y, order = (1, 0, 1))
            ARMAmodel = ARMAmodel.fit(disp=0)

            y_pred = ARMAmodel.get_forecast(3)
            
            y_pred_df = y_pred.conf_int(alpha = 0.05) 
            y_pred_df[attribute] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
            y_pred_df["year"] = [2022, 2023, 2024]
            y_pred_df["suburb"] = i 
            
            if attribute == 'mean_income':
                y_pred_df[attribute] = y_pred_df['upper mean_income']

            full_predictions = pd.concat([full_predictions, y_pred_df[['suburb', attribute , 'year']]])
            
        return full_predictions.reset_index(drop = True)

In [11]:
future_incomes = arma_preds('mean_income')
future_population = arma_preds('population')
future_popden = arma_preds('population_density')
future_unemployment = arma_preds('unemployement')

In [12]:
future_dfs = [future_incomes, future_population, future_popden, future_unemployment]
future_dataframe = pd.concat(future_dfs, join='inner', axis=1)
future_dataframe = future_dataframe.loc[:,~future_dataframe.columns.duplicated()].copy()

future_dataframe

,suburb,mean_income,year,population,population_density,unemployement
0,abbotsford,79318.359206,2022,9783.865576,5621.294297,4.438316
1,abbotsford,82557.279796,2023,9745.402906,5599.184780,4.387938
2,abbotsford,84564.223025,2024,9707.091442,5577.162223,4.338133
3,albert park,125486.441946,2022,16708.118704,3574.162208,2.476385
4,albert park,131698.030678,2023,16698.254643,3572.054658,2.476383
...,...,...,...,...,...,...
454,wodonga,57168.756219,2023,27855.494993,99.907323,3.881209
455,wodonga,57963.754615,2024,27844.747379,99.868718,3.839213
456,yarraville,83099.116938,2022,16454.630937,2927.961689,3.770923
457,yarraville,85668.803193,2023,16451.013238,2927.318918,3.770920


In [13]:
modelling_dataframe = modelling_dataframe.merge(df[['suburb', 'nearest_hospital', 'nearest_train_station',
       'distance_to_cbd', 'nearest_school']], on = 'suburb', how = 'left' )

future_dataframe = future_dataframe.merge(df[['suburb', 'nearest_hospital', 'nearest_train_station',
       'distance_to_cbd', 'nearest_school']], on = 'suburb', how = 'left' )

display(modelling_dataframe.head(10))
display(future_dataframe.head(10))

,suburb,median_rent,year,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school
0,abbotsford,450.0,2012,65938.0,5544.0,3185.291583,5.3,2.934439,1.489185,4.443090,1.067517
1,albert park,470.0,2012,107915.0,15142.0,3239.138340,2.3,44.218611,11.892911,5.411578,0.473433
2,alfredton,270.0,2012,52993.0,9060.0,171.880306,2.8,4.955048,6.510578,121.322730,1.698581
3,alphington,330.0,2012,68272.0,8706.0,3017.363879,4.4,3.177273,1.260436,9.481345,1.014145
4,altona,330.0,2012,56677.0,12361.0,698.965773,4.7,1.313035,0.931396,18.746987,0.597719
5,armadale,385.0,2012,99631.0,9308.0,4262.880696,2.6,2.048613,0.681178,8.733257,0.888396
6,ascot vale,390.0,2012,66592.0,14309.0,3730.090456,7.3,25.070790,1.393727,7.824663,0.737259
7,ashburton,395.0,2012,76336.0,7912.0,2789.254742,4.8,2.938985,0.881708,14.255700,0.654015
8,bairnsdale,260.0,2012,44563.0,13680.0,88.251517,5.8,2.496283,3.384117,280.293767,1.568367
9,balwyn,460.0,2012,82777.0,16319.0,2921.933751,3.5,2.927600,27.261647,11.947300,0.833513


,suburb,mean_income,year,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school
0,abbotsford,79318.359206,2022,9783.865576,5621.294297,4.438316,2.934439,1.489185,4.443090,1.067517
1,abbotsford,82557.279796,2023,9745.402906,5599.184780,4.387938,2.934439,1.489185,4.443090,1.067517
2,abbotsford,84564.223025,2024,9707.091442,5577.162223,4.338133,2.934439,1.489185,4.443090,1.067517
3,albert park,125486.441946,2022,16708.118704,3574.162208,2.476385,44.218611,11.892911,5.411578,0.473433
4,albert park,131698.030678,2023,16698.254643,3572.054658,2.476383,44.218611,11.892911,5.411578,0.473433
5,albert park,135533.443278,2024,16688.396406,3569.948351,2.476381,44.218611,11.892911,5.411578,0.473433
6,alfredton,64464.070193,2022,16830.779254,319.302099,2.379058,4.955048,6.510578,121.322730,1.698581
7,alfredton,66282.866861,2023,16793.611175,318.596878,2.379056,4.955048,6.510578,121.322730,1.698581
8,alfredton,67414.080123,2024,16756.525175,317.893214,2.379053,4.955048,6.510578,121.322730,1.698581
9,alphington,85434.746938,2022,9746.234636,3377.885495,4.858515,3.177273,1.260436,9.481345,1.014145


#### We now have both our train and test set ready ! - Need to Transform Appropiately and Fit Models 

### Step 2 - Applying Appropriate Transformations and Creating Matrices Suitable for The Models 

In [14]:
X_train = modelling_dataframe.drop(['median_rent', 'year'], axis = 1)
X_train = pd.get_dummies(X_train, columns=['suburb'])

y_train = modelling_dataframe['median_rent']

X_train.head()

,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,suburb_abbotsford,suburb_albert park,...,suburb_tullamarine,suburb_vermont,suburb_wangaratta,suburb_wantirna,suburb_warragul,suburb_wendouree,suburb_west footscray,suburb_williamstown,suburb_wodonga,suburb_yarraville
0,65938.0,5544.0,3185.291583,5.3,2.934439,1.489185,4.443090,1.067517,1,0,...,0,0,0,0,0,0,0,0,0,0
1,107915.0,15142.0,3239.138340,2.3,44.218611,11.892911,5.411578,0.473433,0,1,...,0,0,0,0,0,0,0,0,0,0
2,52993.0,9060.0,171.880306,2.8,4.955048,6.510578,121.322730,1.698581,0,0,...,0,0,0,0,0,0,0,0,0,0
3,68272.0,8706.0,3017.363879,4.4,3.177273,1.260436,9.481345,1.014145,0,0,...,0,0,0,0,0,0,0,0,0,0
4,56677.0,12361.0,698.965773,4.7,1.313035,0.931396,18.746987,0.597719,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X_test_2022 = future_dataframe[future_dataframe['year'] == 2022].reset_index(drop = True)
X_test_2022 = X_test_2022.drop(['year'], axis = 1)
suburbs_list = X_test_2022['suburb']
X_test_2022 = pd.get_dummies(X_test_2022, columns=['suburb'])

In [16]:
X_test_2023 = future_dataframe[future_dataframe['year'] == 2023].reset_index(drop = True)
X_test_2023 = X_test_2023.drop(['year'], axis = 1)
X_test_2023 = pd.get_dummies(X_test_2023, columns=['suburb'])

X_test_2023.head()

,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,suburb_abbotsford,suburb_albert park,...,suburb_tullamarine,suburb_vermont,suburb_wangaratta,suburb_wantirna,suburb_warragul,suburb_wendouree,suburb_west footscray,suburb_williamstown,suburb_wodonga,suburb_yarraville
0,82557.279796,9745.402906,5599.184780,4.387938,2.934439,1.489185,4.443090,1.067517,1,0,...,0,0,0,0,0,0,0,0,0,0
1,131698.030678,16698.254643,3572.054658,2.476383,44.218611,11.892911,5.411578,0.473433,0,1,...,0,0,0,0,0,0,0,0,0,0
2,66282.866861,16793.611175,318.596878,2.379056,4.955048,6.510578,121.322730,1.698581,0,0,...,0,0,0,0,0,0,0,0,0,0
3,89247.717750,9742.210779,3376.490850,4.858510,3.177273,1.260436,9.481345,1.014145,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78686.780727,14004.510973,791.899913,4.210875,1.313035,0.931396,18.746987,0.597719,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_test_2024 = future_dataframe[future_dataframe['year'] == 2024].reset_index(drop = True)
X_test_2024 = X_test_2024.drop(['year'], axis = 1)
X_test_2024 = pd.get_dummies(X_test_2024, columns=['suburb'])

X_test_2024.head()

,mean_income,population,population_density,unemployement,nearest_hospital,nearest_train_station,distance_to_cbd,nearest_school,suburb_abbotsford,suburb_albert park,...,suburb_tullamarine,suburb_vermont,suburb_wangaratta,suburb_wantirna,suburb_warragul,suburb_wendouree,suburb_west footscray,suburb_williamstown,suburb_wodonga,suburb_yarraville
0,84564.223025,9707.091442,5577.162223,4.338133,2.934439,1.489185,4.443090,1.067517,1,0,...,0,0,0,0,0,0,0,0,0,0
1,135533.443278,16688.396406,3569.948351,2.476381,44.218611,11.892911,5.411578,0.473433,0,1,...,0,0,0,0,0,0,0,0,0,0
2,67414.080123,16756.525175,317.893214,2.379053,4.955048,6.510578,121.322730,1.698581,0,0,...,0,0,0,0,0,0,0,0,0,0
3,91604.629848,9738.188583,3375.096781,4.858505,3.177273,1.260436,9.481345,1.014145,0,0,...,0,0,0,0,0,0,0,0,0,0
4,80861.977124,14002.383357,791.779633,4.203981,1.313035,0.931396,18.746987,0.597719,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 3 - The Fun Part - Fitting The Model 

### Predicting rents for suburbs in 2022

#### Support Vector Machine

In [18]:
from sklearn.svm import SVR

sv_reg_regressor = SVR()
sv_reg_regressor.fit(X_train, y_train)

y_pred_2022 = sv_reg_regressor.predict(X_test_2022)
y_pred_2023 = sv_reg_regressor.predict(X_test_2023)
y_pred_2024 = sv_reg_regressor.predict(X_test_2024)

sv_reg_predictions = pd.DataFrame(columns = ['Suburb', '2022 Predicted Rent', 
                                                '2023 Predicted Rent', '2024 Predicted Rent'])
sv_reg_predictions['Suburb'] = suburbs_list

sv_reg_predictions['2022 Predicted Rent'] = y_pred_2022
sv_reg_predictions['2023 Predicted Rent'] = y_pred_2023
sv_reg_predictions['2024 Predicted Rent'] = y_pred_2024

sv_reg_predictions

,Suburb,2022 Predicted Rent,2023 Predicted Rent,2024 Predicted Rent
0,abbotsford,421.672417,428.589175,432.691009
1,albert park,474.101118,472.516709,470.804697
2,alfredton,387.612743,391.916478,394.594473
3,alphington,433.772000,441.074879,445.263511
4,altona,412.096521,420.105250,424.889729
...,...,...,...,...
148,wendouree,359.181816,363.079170,365.554850
149,west footscray,381.069892,385.234280,387.839629
150,williamstown,451.457309,455.385712,457.647406
151,wodonga,372.377987,375.146174,376.889261


#### XGboost Regression 

In [19]:
from xgboost.sklearn import XGBRegressor

xgboost_regressor = XGBRegressor()
xgboost_regressor.fit(X_train, y_train)

y_pred_2022 = xgboost_regressor.predict(X_test_2022)
y_pred_2023 = xgboost_regressor.predict(X_test_2023)
y_pred_2024 = xgboost_regressor.predict(X_test_2024)

predictions = pd.DataFrame(columns = ['Suburb', '2022 Predicted Rent', 
                                           '2023 Predicted Rent', '2024 Predicted Rent'])
predictions['Suburb'] = suburbs_list
predictions['2022 Predicted Rent'] = y_pred_2022
predictions['2023 Predicted Rent'] = y_pred_2023
predictions['2024 Predicted Rent'] = y_pred_2024

predictions

,Suburb,2022 Predicted Rent,2023 Predicted Rent,2024 Predicted Rent
0,abbotsford,497.688965,522.485413,518.077332
1,albert park,552.319214,532.987732,537.693787
2,alfredton,372.218994,374.568787,408.667297
3,alphington,406.877350,412.279083,412.279083
4,altona,420.777618,414.271790,414.646149
...,...,...,...,...
148,wendouree,339.344910,335.708374,346.656311
149,west footscray,379.167267,377.765472,392.516998
150,williamstown,504.787842,506.743896,510.728485
151,wodonga,323.677551,323.896454,323.167023


#### Elastic Net

In [20]:
from sklearn.linear_model import ElasticNet


elasticnet_regressor = ElasticNet()
elasticnet_regressor.fit(X_train, y_train)

y_pred_2022 = elasticnet_regressor.predict(X_test_2022)
y_pred_2023 = elasticnet_regressor.predict(X_test_2023)
y_pred_2024 = elasticnet_regressor.predict(X_test_2024)

elasticnet_predictions = pd.DataFrame(columns = ['Suburb', '2022 Predicted Rent', 
                                                '2023 Predicted Rent', '2024 Predicted Rent'])
elasticnet_predictions['Suburb'] = suburbs_list

elasticnet_predictions['2022 Predicted Rent'] = y_pred_2022
elasticnet_predictions['2023 Predicted Rent'] = y_pred_2023
elasticnet_predictions['2024 Predicted Rent'] = y_pred_2024

elasticnet_predictions

,Suburb,2022 Predicted Rent,2023 Predicted Rent,2024 Predicted Rent
0,abbotsford,448.720529,454.339322,457.833113
1,albert park,541.922780,552.610630,559.202770
2,alfredton,356.624024,359.725789,361.642466
3,alphington,443.295815,449.857632,453.909702
4,altona,412.240980,418.338784,422.110650
...,...,...,...,...
148,wendouree,311.723077,314.913477,316.901118
149,west footscray,393.827956,396.876878,398.770183
150,williamstown,460.479663,464.920681,467.681557
151,wodonga,247.241833,249.585240,251.098376
